In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import numpy as np
import tensorflow as tf
from scipy.misc import imread, imresize, imsave, imshow
# from imagenet_classes import class_names 
import matplotlib.pyplot as plt
import json
import pickle
import os
import re



%matplotlib inline

# Import

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Clean

In [3]:
#  I need to reduce labels on mnist dataset for testing each pass

In [4]:
def keep_n_labels(passed_data, n):
    data = np.copy(passed_data)
    all_indexes = np.asarray(list(range(len(data))))
    np.random.shuffle(all_indexes)
    if n >= 1:
        selected_indexes_to_set_to_0 = np.asarray(list(all_indexes[:-n]))
    else:
        selected_indexes_to_set_to_0 = np.asarray(list(all_indexes[:]))
#     normal_indexes = np.asarray(list(all_indexes[-n:]))
    data[selected_indexes_to_set_to_0] = np.zeros(data.shape[1])
    return data


In [5]:
mnist_train_y_0_labeled = keep_n_labels(mnist.train.labels, 0)
mnist_train_y_10_labeled = keep_n_labels(mnist.train.labels, 10)
mnist_train_y_50_labeled = keep_n_labels(mnist.train.labels, 50)
mnist_train_y_100_labeled = keep_n_labels(mnist.train.labels, 100)
mnist_train_y_200_labeled = keep_n_labels(mnist.train.labels, 200)
mnist_train_y_500_labeled = keep_n_labels(mnist.train.labels, 500)
mnist_train_y_1000_labeled = keep_n_labels(mnist.train.labels, 1000)
mnist_train_y_5000_labeled = keep_n_labels(mnist.train.labels, 5000)
mnist_train_y_10000_labeled = keep_n_labels(mnist.train.labels, 10000)
mnist_train_y_all_labeled = np.copy(mnist.train.labels)



print(np.sum(mnist_train_y_0_labeled))
print(np.sum(mnist_train_y_50_labeled))
print(np.sum(mnist_train_y_500_labeled))
print(np.sum(mnist_train_y_10_labeled))
print(np.sum(mnist_train_y_10000_labeled))
print(np.sum(mnist_train_y_all_labeled))


0.0
50.0
500.0
10.0
10000.0
55000.0


In [6]:
"""Define get next batch method"""
current_index = 0
current_data_shuffle_mat = None
def get_next_batch(data, classes, num_to_select):
    global current_index, current_data_shuffle_mat
    #  shuffle with labels
    if current_data_shuffle_mat is None or current_index + num_to_select >= len(data):
        current_data_shuffle_mat =  np.asarray(zip(data, classes))
        np.random.shuffle(current_data_shuffle_mat)
        current_index = 0
    
    data_mat_selected = np.asarray(list(\
                        current_data_shuffle_mat[current_index: current_index + num_to_select,0]))
    data_classes_selected = np.asarray(list(\
                        current_data_shuffle_mat[current_index: current_index + num_to_select,1]))
    current_index += num_to_select
    
#     return data_mat_selected.astype(float), data_classes_selected.astype(float)
    return data_mat_selected, data_classes_selected




%timeit -n 10000 get_next_batch(mnist.train.images, mnist_train_y_10000_labeled, 2)
10000 loops, best of 3: 11.1 µs per loop
%timeit -n 100 get_next_batch_orig(mnist.train.images, mnist_train_y_10000_labeled, 2)
100 loops, best of 3: 71.4 ms per loop
%timeit -n 10000 mnist.train.next_batch(2)
10000 loops, best of 3: 1.59 µs per loop

#  Import / Clean Sleep Cycle Data

In [7]:
is_5_mins = True

train_original_df = pd.read_csv('/*****')
#  choose validation set or test set
valid_original_df = pd.read_csv('/*****')


In [8]:
print(train_original_df.info())
train_original_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Columns: 609 entries, time to userId
dtypes: float64(607), int64(1), object(1)
memory usage: 47.1+ MB
None


,time,hr0,hr1,hr2,hr3,hr4,hr5,hr6,hr7,hr8,...,ox298,ox299,Invalid,S4,S3,S2,S1,REM,W,userId
0,0,84.013123,83.012131,82.011139,80.009155,80.009155,81.010147,82.011139,82.011139,83.012131,...,97.001602,98.002594,0.003333,0.0,0.0,0.0,0.0,0.0,0.996667,ins2
1,1,69.010452,70.011444,70.011444,70.011444,70.011444,70.011444,69.010452,70.011444,69.010452,...,97.001602,97.001602,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,ins2
2,2,77.006180,76.005188,77.006180,76.005188,77.006180,77.006180,77.006180,77.006180,78.007172,...,97.001602,97.001602,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,ins2
3,3,75.016403,75.016403,75.016403,76.005188,75.016403,75.016403,75.016403,75.016403,76.005188,...,97.001602,97.001602,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,ins2
4,4,72.013428,72.013428,73.014420,72.013428,73.014420,72.013428,73.014420,73.014420,72.013428,...,97.001602,97.001602,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,ins2


In [9]:
print(valid_original_df.info())
valid_original_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Columns: 609 entries, time to userId
dtypes: float64(607), int64(1), object(1)
memory usage: 5.8+ MB
None


,time,hr0,hr1,hr2,hr3,hr4,hr5,hr6,hr7,hr8,...,ox298,ox299,Invalid,S4,S3,S2,S1,REM,W,userId
0,0,83.076219,82.075227,82.075227,81.074235,81.074235,81.074235,81.074235,81.074235,81.074235,...,95.017929,95.017929,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8
1,1,88.068971,88.068971,88.068971,88.068971,87.080186,86.079194,83.076219,81.074235,80.073243,...,94.018463,94.018463,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8
2,2,75.083543,75.083543,75.083543,75.083543,74.082551,74.082551,74.082551,73.081559,73.081559,...,93.023575,93.023575,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8
3,3,73.081559,74.082551,74.082551,75.083543,75.083543,75.083543,75.083543,74.082551,73.081559,...,93.023575,94.018463,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8
4,4,78.071260,78.071260,79.072251,79.072251,79.072251,79.072251,79.072251,79.072251,80.073243,...,95.017929,95.017929,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8


In [10]:
def mean_center_data(passed_df):
    """Takes a train/test/validation set in format time, hr, ox, classes, userId and mean centered the
    data for each user"""
    
    #  find mean ox and hr
    mean_values_by_userId = []

    for name, group in passed_df.groupby('userId'):  
        if is_5_mins:
            mean_hr = group.loc[:, 'hr0':'hr299'].as_matrix().mean()
            mean_ox = group.loc[:, 'ox0':'ox299'].as_matrix().mean()
        else:
            mean_hr = group.loc[:, 'hr0':'hr59'].as_matrix().mean()
            mean_ox = group.loc[:, 'ox0':'ox59'].as_matrix().mean()

        mean_values_by_userId.append([name, mean_hr, mean_ox])
        
        
    #  convert to dataframe
    mean_values_by_userId = pd.DataFrame(mean_values_by_userId)
    mean_values_by_userId.columns = ['userId', 'mean_hr', 'mean_ox']
        
        
    #  merge with training/etc data, then mean center, then drop the mean values columns
    mean_centered_df = pd.merge(passed_df, mean_values_by_userId, on='userId', how='left')
    if is_5_mins:
        mean_centered_df.loc[:, 'hr0':'hr299'] = mean_centered_df.loc[:, 'hr0':'hr299'].as_matrix() - \
                                                np.atleast_2d(mean_centered_df.loc[:, 'mean_hr'].as_matrix()).T
        mean_centered_df.loc[:, 'ox0':'ox299'] = mean_centered_df.loc[:, 'ox0':'ox299'].as_matrix() - \
                                                np.atleast_2d(mean_centered_df.loc[:, 'mean_ox'].as_matrix()).T
    else:
        mean_centered_df.loc[:, 'hr0':'hr59'] = mean_centered_df.loc[:, 'hr0':'hr59'].as_matrix() - \
                                                np.atleast_2d(mean_centered_df.loc[:, 'mean_hr'].as_matrix()).T
        mean_centered_df.loc[:, 'ox0':'ox59'] = mean_centered_df.loc[:, 'ox0':'ox59'].as_matrix() - \
                                                np.atleast_2d(mean_centered_df.loc[:, 'mean_ox'].as_matrix()).T


    mean_centered_df.drop(['mean_hr','mean_ox'], axis=1, inplace=True)
    return mean_centered_df

In [11]:
mean_centered_train_df = mean_center_data(train_original_df)
mean_centered_valid_df = mean_center_data(valid_original_df)

In [12]:
mean_centered_valid_df.head(2)

,time,hr0,hr1,hr2,hr3,hr4,hr5,hr6,hr7,hr8,...,ox298,ox299,Invalid,S4,S3,S2,S1,REM,W,userId
0,0,18.389260,17.388268,17.388268,16.387276,16.387276,16.387276,16.387276,16.387276,16.387276,...,2.978348,2.978348,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8
1,1,23.382012,23.382012,23.382012,23.382012,22.393227,21.392235,18.389260,16.387276,15.386284,...,1.978882,1.978882,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8


In [13]:
mean_centered_valid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1253 entries, 0 to 1252
Columns: 609 entries, time to userId
dtypes: float64(607), int64(1), object(1)
memory usage: 5.8+ MB


In [14]:
mean_centered_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10127 entries, 0 to 10126
Columns: 609 entries, time to userId
dtypes: float64(607), int64(1), object(1)
memory usage: 47.1+ MB


In [15]:
#  find majority class labels


#  remove class percent columns (6) at the end


#  this section accounts for invalid entries. First it actively saves all valid classes as
#  last_valid_class, then if any valid classes are encountered they are replaced with this 
#  last valid class value
#  all users are linked together so values might be taken from the user before if they start out invalid
#  this should be fairly infrequent though and to boot most users start and end in the wake state so it
#  should be alright to link their values back to back like this



last_valid_class = 0  #  this is the wake class, it's a fair to assume most start in the wake phase

#  majority classes for train
majority_classes_train = []
for row in mean_centered_train_df.iloc[:,-8:-1].as_matrix():
    #  4 classes DEEP(S4/S3), LIGHT(S2/S1), REM, W
#     row = [row[0], row[1] + row[2], row[3] + row[4], row[5], row[6]]
    #  3 classes DEEP, LIGHT (including REM), Wake
    row = [row[0], row[1] + row[2], row[3] + row[4] + row[5], row[6]]
    #  2 classes SLEEP, Wake
#     row = [row[0], row[1] + row[2] + row[3] + row[4] + row[5], row[6]]


    current_class = np.argmax(row) - 1
    if current_class < 0:
        current_class = last_valid_class
    else:
        last_valid_class = current_class
    majority_classes_train.append(current_class)



last_valid_class = 0  #  this is the wake class, it's a fair to assume most start in the wake phase

#  majority classes for train
majority_classes_valid = []
for row in mean_centered_valid_df.iloc[:,-8:-1].as_matrix():
    #  4 classes DEEP(S4/S3), LIGHT(S2/S1), REM, W
#     row = [row[0], row[1] + row[2], row[3] + row[4], row[5], row[6]]
    # 3 classes DEEP, LIGHT (including REM), Wake
    row = [row[0], row[1] + row[2], row[3] + row[4] + row[5], row[6]]
    #  2 classes SLEEP, Wake
#     row = [row[0], row[1] + row[2] + row[3] + row[4] + row[5], row[6]]

    
    current_class = np.argmax(row) - 1
    if current_class < 0:
        current_class = last_valid_class
    else:
        last_valid_class = current_class
    majority_classes_valid.append(current_class)



In [16]:
#  check to make sure conversion worked (should see 3 for d,l,w etc)
np.bincount(majority_classes_train)

array([1800, 6279, 2048])

In [17]:
#  extract stardize shuffle

#  extract data for train/valid

#  include time (in n minute instances) since sleep began
if is_5_mins:
    train_original_mat = mean_centered_train_df.loc[:,'time':'ox299'].as_matrix()
    valid_original_mat = mean_centered_valid_df.loc[:,'time':'ox299'].as_matrix()
else:
    train_original_mat = mean_centered_train_df.loc[:,'time':'ox59'].as_matrix()
    valid_original_mat = mean_centered_valid_df.loc[:,'time':'ox59'].as_matrix()
# train_original_mat = mean_centered_train_df.loc[:,'hr0':'ox299'].as_matrix()
# valid_original_mat = mean_centered_valid_df.loc[:,'hr0':'ox299'].as_matrix()

train_original_mat

array([[  0.        ,  13.26134914,  12.26035714, ...,   0.79259893,
          0.79259893,   1.79359093],
       [  1.        ,  -1.74132186,  -0.74032986, ...,   0.79259893,
          0.79259893,   0.79259893],
       [  2.        ,   6.25440614,   5.25341414, ...,   0.79259893,
          0.79259893,   0.79259893],
       ..., 
       [ 93.        ,  10.38672983,  10.38672983, ...,   1.5138803 ,
          1.5138803 ,   1.5138803 ],
       [ 94.        ,   1.54526583,   0.63063183, ...,   0.6602223 ,
          0.6602223 ,   0.6602223 ],
       [ 95.        ,   9.47209483,  10.08185083, ...,   0.5382703 ,
          0.6602223 ,   0.6602223 ]])

In [18]:
#  standardize
scaler = StandardScaler()
scaler.fit(train_original_mat)
train_std_mat = scaler.transform(train_original_mat)
valid_std_mat = scaler.transform(valid_original_mat)

In [19]:
#  shuffle with labels
train_shuffled_mat =  np.asarray(zip(train_std_mat, majority_classes_train))
np.random.shuffle(train_shuffled_mat)
valid_shuffled_mat =  np.asarray(zip(valid_std_mat, majority_classes_valid))
np.random.shuffle(valid_shuffled_mat)

train_mat = np.asarray(list(train_shuffled_mat[:,0]))
train_classes = train_shuffled_mat[:,1]
valid_mat = np.asarray(list(valid_shuffled_mat[:,0]))
valid_classes = valid_shuffled_mat[:,1]

In [20]:
#  one hot encoded classes in case I need them

#  TODO:  change this for OHE 6 , 4, then 3, 2

train_classes_ohe = np.zeros((len(train_classes),3))
for i,c in enumerate(train_classes):
    train_classes_ohe[i,c] = 1.0
    
valid_classes_ohe = np.zeros((len(valid_classes),3))
for i,c in enumerate(valid_classes):
    valid_classes_ohe[i,c] = 1.0

In [21]:
train_classes_ohe

array([[ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       ..., 
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.]])

In [22]:
train_std_mat.shape

(10127, 601)

In [23]:
sleep_train_y_0_labeled = keep_n_labels(train_classes_ohe, 0)
sleep_train_y_10_labeled = keep_n_labels(train_classes_ohe, 10)
sleep_train_y_50_labeled = keep_n_labels(train_classes_ohe, 50)
sleep_train_y_100_labeled = keep_n_labels(train_classes_ohe, 100)
sleep_train_y_200_labeled = keep_n_labels(train_classes_ohe, 200)
sleep_train_y_500_labeled = keep_n_labels(train_classes_ohe, 500)
sleep_train_y_1000_labeled = keep_n_labels(train_classes_ohe, 1000)
sleep_train_y_5000_labeled = keep_n_labels(train_classes_ohe, 5000)
sleep_train_y_10000_labeled = keep_n_labels(train_classes_ohe, 10000)
sleep_train_y_all_labeled = np.copy(train_classes_ohe)



print(np.sum(sleep_train_y_0_labeled))
print(np.sum(sleep_train_y_50_labeled))
print(np.sum(sleep_train_y_500_labeled))
print(np.sum(sleep_train_y_10_labeled))
print(np.sum(sleep_train_y_10000_labeled))
print(np.sum(sleep_train_y_all_labeled))


0.0
50.0
500.0
10.0
10000.0
10127.0


#  CNN model SLEEP

In [24]:
sess = tf.InteractiveSession()    
   
batch_size = 50
    

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def max_pool_4x4(x):
  return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                        strides=[1, 4, 4, 1], padding='SAME')

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')    
    
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=1e-4)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)








with tf.name_scope('Wx_B') as scope:

    
    x = tf.placeholder(tf.float32, shape=[None, 560])
    x_batch_size = tf.shape(x)[0]
    y_ = tf.placeholder(tf.float32, shape=[None, 3])
    x_image = tf.reshape(x, [-1,20,28,1])
    #  this truncates the last value
#     x_image = tf.image.resize_image_with_crop_or_pad(x_image_ut, 20,30) 
    
    
#     x = tf.placeholder(tf.float32, shape=[None, 784])
#     x_batch_size = tf.shape(x)[0]
#     y_ = tf.placeholder(tf.float32, shape=[None, 10])
#     x_image = tf.reshape(x, [-1,28,28,1])

    W_conv1_bnn1 = weight_variable([5, 5, 1, 32])
    b_conv1_bnn1 = bias_variable([32])
    h_conv1_bnn1 = tf.nn.relu(conv2d(x_image, W_conv1_bnn1) + b_conv1_bnn1)
    h_pool1_bnn1 = max_pool_2x2(h_conv1_bnn1)

    W_conv2_bnn1 = weight_variable([5, 5, 32, 40])
    b_conv2_bnn1 = bias_variable([40])
    h_conv2_bnn1 = tf.nn.relu(conv2d(h_pool1_bnn1, W_conv2_bnn1) + b_conv2_bnn1)
    h_pool2_bnn1 = max_pool_2x2(h_conv2_bnn1)

    W_conv3_bnn1 = weight_variable([5, 5, 40, 16])
    b_conv3_bnn1 = bias_variable([16])
    h_conv3_bnn1 = tf.nn.relu(conv2d(h_pool2_bnn1, W_conv3_bnn1) + b_conv3_bnn1)
    #     h_pool3 = max_pool_2x2(h_conv3)

    W_fc1_bnn1 = weight_variable([5 * 7 * 16, 100])
    b_fc1_bnn1 = bias_variable([100])
    h_conv3_flat_bnn1 = tf.reshape(h_conv3_bnn1, [-1, 5*7*16])
    h_fc1_bnn1 = tf.nn.relu(tf.matmul(h_conv3_flat_bnn1, W_fc1_bnn1) + b_fc1_bnn1)


    keep_prob = tf.placeholder(tf.float32)
    y_raw_bnn1 = h_fc1_bnn1

        
        
    
    
    #  final fc layer transformation to 10 classes
    W_fc3 = weight_variable([100, 3])
    b_fc3 = bias_variable([3])
    y_fc3 = tf.matmul(y_raw_bnn1, W_fc3) + b_fc3

    
    y_conv=tf.nn.softmax(y_fc3)
    out_shape = tf.shape(y_conv)
    print(y_conv.shape)

    



    
    
def l2_loss(tensor_inst):
    return tf.reduce_sum(tensor_inst ** 2)
beta = 0.0001

with tf.name_scope('cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1])) + \
                beta*l2_loss(W_conv1_bnn1) +\
                beta*l2_loss(b_conv1_bnn1) +\
                beta*l2_loss(W_conv2_bnn1) +\
                beta*l2_loss(b_conv2_bnn1) +\
                beta*l2_loss(W_conv3_bnn1) +\
                beta*l2_loss(b_conv3_bnn1) +\
                beta*l2_loss(W_fc1_bnn1) +\
                beta*l2_loss(b_fc1_bnn1) +\
                beta*l2_loss(W_fc3) +\
                beta*l2_loss(b_fc3)
    loss = cross_entropy


with tf.name_scope('optimizer') as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))






#  summary writers for debugging
summary_writer = tf.summary.FileWriter( \
 '/home/tuna/Projects/CS678/Ladder_Networks_Project_1/tf_logs/cnn_sleep_100_labels/v1',\
                                    graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', loss )
merged_summary_op = tf.summary.merge_all()
#  additional for seperate runs
valid_acc_summary = tf.summary.scalar( 'validation_accuracy', accuracy )
test_acc_summary = tf.summary.scalar( 'test_accuracy', accuracy )

sess.run(tf.global_variables_initializer())



for i in range(5000):
#     batch = mnist.train.next_batch(batch_size)
    batch_xs, batch_ys = get_next_batch(train_mat[:,:-41], sleep_train_y_10_labeled, batch_size)

#     if i < 1000:
#         _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: batch_ys,
#                                                                           isTrain: True, isLabeled: True})
#     else:
#         _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: np.zeros(batch_ys.shape, dtype=np.float32),
#                                                                           isTrain: True, isLabeled: True})
#     _, summary_ = sess.run([train_step, merged_summary_op], feed_dict={x: batch[0], y_: np.zeros(batch[1].shape, dtype=np.float32),
#                                                                       isTrain: True, isLabeled: False})
    _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: batch_ys})
    

    summary_writer.add_summary(summary_, i)
    if i%100 == 0:
        #  print training loss
        print("step %d, train_loss %g" % (i, train_loss))
        
        #  validation set results
        acc, ls, valid_summary = sess.run([accuracy, loss, valid_acc_summary], \
                                        feed_dict={x: valid_mat[:,:-41], y_: valid_classes_ohe})
        summary_writer.add_summary(valid_summary, i)
        print("step %d, validation accuracy %g, loss %g" % (i, acc, ls))

        
#  test set results
acc, test_acc_summary = sess.run([accuracy, test_acc_summary], feed_dict={x: valid_mat[:,:-41], y_: valid_classes_ohe})
summary_writer.add_summary(test_acc_summary, 1)
summary_writer.add_summary(test_acc_summary, 100)
summary_writer.add_summary(test_acc_summary, 1000)
summary_writer.add_summary(test_acc_summary, 10)
print("test accuracy %g" % acc)



summary_writer.close()
sess.close()

(?, 3)
step 0, train_loss 0.0221637
step 0, validation accuracy 0.54589, loss 1.09834
step 100, train_loss 0.000156121
step 100, validation accuracy 0.54589, loss 1.096
step 200, train_loss 0.000126327
step 200, validation accuracy 0.54589, loss 1.09506
step 300, train_loss 0.000102204
step 300, validation accuracy 0.54589, loss 1.09261
step 400, train_loss 8.36575e-05
step 400, validation accuracy 0.54589, loss 1.09217
step 500, train_loss 7.45613e-05
step 500, validation accuracy 0.54589, loss 1.08699
step 600, train_loss 5.38167e-05
step 600, validation accuracy 0.54589, loss 1.09049
step 700, train_loss 4.61535e-05
step 700, validation accuracy 0.54589, loss 1.08931
step 800, train_loss 3.9487e-05
step 800, validation accuracy 0.54589, loss 1.08748
step 900, train_loss 3.84626e-05
step 900, validation accuracy 0.54589, loss 1.08354
step 1000, train_loss 3.27797e-05
step 1000, validation accuracy 0.54589, loss 1.085
step 1100, train_loss 2.86436e-05
step 1100, validation accuracy 0.

In [25]:
train_mat[:,:-41].shape

(10127, 560)

In [37]:
xs, ys =  get_next_batch(train_mat[:,:-25], sleep_train_y_all_labeled, batch_size)
xs.shape

(50, 576)

(50, 600)

In [28]:
train_mat[:,:-1].shape

(10127, 600)

#  Baseline Model SLEEP

In [24]:
sess = tf.InteractiveSession()    
   
batch_size = 50
    

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=1e-4)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)








with tf.name_scope('Wx_B') as scope:

    x = tf.placeholder(tf.float32, shape=[None, 601])
    x_batch_size = tf.shape(x)[0]
    y_ = tf.placeholder(tf.float32, shape=[None, 3])
    x_image = tf.reshape(x, [-1,601])
        
        
    #  final fc layer transformation to 10 classes
    W_fc3 = weight_variable([601, 3])
    b_fc3 = bias_variable([3])
    y_fc3 = tf.matmul(x_image, W_fc3) + b_fc3

    
    y_conv=tf.nn.softmax(y_fc3)
    out_shape = tf.shape(y_conv)
    print(y_conv.shape)

    




with tf.name_scope('cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
    loss = cross_entropy


with tf.name_scope('optimizer') as scope:
    train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))






#  summary writers for debugging
summary_writer = tf.summary.FileWriter( \
 '/home/tuna/Projects/CS678/Ladder_Networks_Project_1/tf_logs/baseline_sleep_10_labels/v0',\
                                    graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', loss )
merged_summary_op = tf.summary.merge_all()
#  additional for seperate runs
valid_acc_summary = tf.summary.scalar( 'validation_accuracy', accuracy )
test_acc_summary = tf.summary.scalar( 'test_accuracy', accuracy )

sess.run(tf.global_variables_initializer())



for i in range(15000):
#     batch = mnist.train.next_batch(batch_size)
    batch_xs, batch_ys = get_next_batch(train_mat, sleep_train_y_10_labeled, batch_size)

#     if i < 1000:
#         _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: batch_ys,
#                                                                           isTrain: True, isLabeled: True})
#     else:
#         _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: np.zeros(batch_ys.shape, dtype=np.float32),
#                                                                           isTrain: True, isLabeled: True})
#     _, summary_ = sess.run([train_step, merged_summary_op], feed_dict={x: batch[0], y_: np.zeros(batch[1].shape, dtype=np.float32),
#                                                                       isTrain: True, isLabeled: False})
    _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: batch_ys})
    

    summary_writer.add_summary(summary_, i)
    if i%100 == 0:
        #  print training loss
        print("step %d, train_loss %g" % (i, train_loss))
        
        #  validation set results
        acc, ls, valid_summary = sess.run([accuracy, loss, valid_acc_summary], \
                                        feed_dict={x: valid_mat, y_: valid_classes_ohe})
        summary_writer.add_summary(valid_summary, i)
        print("step %d, validation accuracy %g, loss %g" % (i, acc, ls))

        
#  test set results
acc, test_acc_summary = sess.run([accuracy, test_acc_summary], feed_dict={x: valid_mat, y_: valid_classes_ohe})
summary_writer.add_summary(test_acc_summary, 1)
summary_writer.add_summary(test_acc_summary, 100)
summary_writer.add_summary(test_acc_summary, 1000)
summary_writer.add_summary(test_acc_summary, 10)
print("test accuracy %g" % acc)



summary_writer.close()
sess.close()

(?, 3)
step 0, train_loss 0
step 0, validation accuracy 0.285714, loss 1.09867
step 100, train_loss 0
step 100, validation accuracy 0.281724, loss 1.09865
step 200, train_loss 0
step 200, validation accuracy 0.297686, loss 1.0986
step 300, train_loss 0
step 300, validation accuracy 0.313647, loss 1.09857
step 400, train_loss 0
step 400, validation accuracy 0.351955, loss 1.09853
step 500, train_loss 0
step 500, validation accuracy 0.39585, loss 1.09849
step 600, train_loss 0
step 600, validation accuracy 0.438947, loss 1.09845
step 700, train_loss 0
step 700, validation accuracy 0.453312, loss 1.09842
step 800, train_loss 0
step 800, validation accuracy 0.475658, loss 1.09838
step 900, train_loss 0
step 900, validation accuracy 0.477255, loss 1.09837
step 1000, train_loss 0
step 1000, validation accuracy 0.481245, loss 1.09831
step 1100, train_loss 0
step 1100, validation accuracy 0.477255, loss 1.09826
step 1200, train_loss 0
step 1200, validation accuracy 0.47486, loss 1.09823
step 1